In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [14]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)


/Users/spencer/Desktop/Repos/nba-predictor/preprocessing.py:43: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['matchup', 'team_abbreviation', 'team_name', 'wl'], dtype='object')
  rolling_averages = grouped[columns_to_average].apply(lambda x: x.rolling(window=len(x), min_periods=1).mean().shift(1))
/Users/spencer/Desktop/Repos/nba-predictor/preprocessing.py:43: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rolling_averages = grouped[columns_to_average].apply(lambda x: x.rolling

In [15]:

X_train = X_train.values.reshape(-1, 1, 135)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 135)    # Reshape to (N, 1, 136), where N is the number of test samples

train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [52]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out


In [28]:
input_size = X_train.shape[2]  # The number of features in your preprocessed data
hidden_size1 = 64
hidden_size2 = 32
output_size = 2  # Win or loss (binary classification)

model = MLPModel(input_size, hidden_size1, hidden_size2, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [54]:
num_epochs = 50
total_loss = []
acc = []
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.squeeze(1).float(), labels.long()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.squeeze(1).float(), labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        total_loss.append(loss.item())
        acc.append(100 * correct / total)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Acc: {100 * correct / total:.2f}%')



Epoch [1/50], Loss: 0.5668, Validation Acc: 65.07%
Epoch [2/50], Loss: 0.5636, Validation Acc: 64.65%
Epoch [3/50], Loss: 0.5287, Validation Acc: 65.45%
Epoch [4/50], Loss: 0.5846, Validation Acc: 65.04%
Epoch [5/50], Loss: 0.5113, Validation Acc: 65.27%
Epoch [6/50], Loss: 0.5086, Validation Acc: 65.20%
Epoch [7/50], Loss: 0.4475, Validation Acc: 65.16%
Epoch [8/50], Loss: 0.6149, Validation Acc: 65.14%
Epoch [9/50], Loss: 0.6208, Validation Acc: 64.35%
Epoch [10/50], Loss: 0.6565, Validation Acc: 65.40%
Epoch [11/50], Loss: 0.4662, Validation Acc: 65.14%
Epoch [12/50], Loss: 0.5606, Validation Acc: 65.05%
Epoch [13/50], Loss: 0.5088, Validation Acc: 64.82%
Epoch [14/50], Loss: 0.5310, Validation Acc: 65.12%
Epoch [15/50], Loss: 0.5331, Validation Acc: 65.05%
Epoch [16/50], Loss: 0.4471, Validation Acc: 64.94%
Epoch [17/50], Loss: 0.6024, Validation Acc: 64.81%
Epoch [18/50], Loss: 0.5908, Validation Acc: 65.26%
Epoch [19/50], Loss: 0.4599, Validation Acc: 64.05%
Epoch [20/50], Loss: 

In [55]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.squeeze(1).float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 65.21%


In [34]:
# # Export to ONNX
# example_input = torch.tensor(X_train[0]).float()  # Take a sample input from your dataset
# example_input = example_input.to(model.fc1.weight.dtype)

# torch.onnx.export(
#     model,
#     example_input,
#     "models/MLP.onnx",
#     input_names=["input"],
#     output_names=["output"]
# )

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [47]:
def objective(trial):
    # define the hyperparameters to tune
    input_size = X_train.shape[2]
    hidden_size1 = trial.suggest_int("hidden_size1", 16, 128, log=True)
    hidden_size2 = trial.suggest_int("hidden_size2", 16, 128, log=True)
    output_size = 2

    # create the model with the given hyperparameters
    model = MLPModel(input_size, hidden_size1, hidden_size2, output_size)

    # define the optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float("lr", 1e-4, 1e-2, log=True))
    criterion = nn.CrossEntropyLoss()

    num_epochs = 50
    # train the model
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.squeeze(1).float(), labels.long()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in valid_loader:
                inputs, labels = inputs.squeeze(1).float(), labels.long()
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            trial.report(100 * correct / total, epoch)

            # early stopping
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return 100 * correct / total

In [48]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# print the best hyperparameters and the best validation accuracy
print("Best hyperparameters: ", study.best_params)
print("Best validation accuracy: ", study.best_value)


[I 2023-05-03 02:33:04,456] A new study created in memory with name: no-name-c09d6e7b-6910-40e0-bbf7-96afb0db48ae
[I 2023-05-03 02:34:10,842] Trial 0 finished with value: 63.838283004741704 and parameters: {'hidden_size1': 94, 'hidden_size2': 19, 'lr': 0.00705872930557031}. Best is trial 0 with value: 63.838283004741704.
[I 2023-05-03 02:35:23,137] Trial 1 finished with value: 61.330172198652356 and parameters: {'hidden_size1': 80, 'hidden_size2': 51, 'lr': 0.002638216011292618}. Best is trial 0 with value: 63.838283004741704.
[I 2023-05-03 02:36:30,397] Trial 2 finished with value: 61.84177689044173 and parameters: {'hidden_size1': 30, 'hidden_size2': 58, 'lr': 0.002658352821377608}. Best is trial 0 with value: 63.838283004741704.
[I 2023-05-03 02:37:53,079] Trial 3 finished with value: 61.15547791365111 and parameters: {'hidden_size1': 84, 'hidden_size2': 26, 'lr': 0.0007860723359471324}. Best is trial 0 with value: 63.838283004741704.
[I 2023-05-03 02:39:36,250] Trial 4 finished wit

Best hyperparameters:  {'hidden_size1': 16, 'hidden_size2': 96, 'lr': 0.00948339410148986}
Best validation accuracy:  65.95957075118542


In [50]:
input_size = X_train.shape[2]  # The number of features in your preprocessed data
hidden_size1 = 16
hidden_size2 = 96
output_size = 2  # Win or loss (binary classification)

model = MLPModel(input_size, hidden_size1, hidden_size2, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00948339410148986)


In [56]:
num_epochs = 50
total_loss = []
acc = []
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.squeeze(1).float(), labels.long()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.squeeze(1).float(), labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        total_loss.append(loss.item())
        acc.append(100 * correct / total)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Acc: {100 * correct / total:.2f}%')



Epoch [1/50], Loss: 0.6311, Validation Acc: 64.74%
Epoch [2/50], Loss: 0.4440, Validation Acc: 64.76%
Epoch [3/50], Loss: 0.5627, Validation Acc: 65.05%
Epoch [4/50], Loss: 0.4774, Validation Acc: 64.70%
Epoch [5/50], Loss: 0.3900, Validation Acc: 64.60%
Epoch [6/50], Loss: 0.4982, Validation Acc: 64.94%
Epoch [7/50], Loss: 0.5794, Validation Acc: 65.04%
Epoch [8/50], Loss: 0.4394, Validation Acc: 65.22%
Epoch [9/50], Loss: 0.5021, Validation Acc: 65.16%
Epoch [10/50], Loss: 0.5572, Validation Acc: 65.07%
Epoch [11/50], Loss: 0.5900, Validation Acc: 64.81%
Epoch [12/50], Loss: 0.5473, Validation Acc: 65.04%
Epoch [13/50], Loss: 0.5668, Validation Acc: 65.21%
Epoch [14/50], Loss: 0.4651, Validation Acc: 64.86%
Epoch [15/50], Loss: 0.4718, Validation Acc: 64.64%
Epoch [16/50], Loss: 0.4877, Validation Acc: 64.89%
Epoch [17/50], Loss: 0.5014, Validation Acc: 65.11%
Epoch [18/50], Loss: 0.5274, Validation Acc: 65.21%
Epoch [19/50], Loss: 0.5068, Validation Acc: 65.17%
Epoch [20/50], Loss: 